# ***Byte-Level vs. Token-level Robustness: Comparing Byt5 and T5 under translation of misspellings***

Group: 84  
Students: s252133, s252085, s252086, s251590


## ***Imports***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import ast
from collections import Counter
import sacrebleu



ModuleNotFoundError: No module named 'sacrebleu'

## ***Load datasets***

In [3]:
byt5_translated = pd.read_csv("./data/byt5_translated.csv")
byt5_translated.head(3)


,en,es
0,I want to talk to you today a little bit about...,['Hoy quiero hablarles un poco de la irraciona...
1,I was burned very badly.And if you spend a lot...,['Me quemé muy mal.Y si pasan mucho tiempo en ...
2,The nurses in my department thought that the r...,['Las enfermeras en mi departamento pensaban q...


In [4]:
t5_translated = pd.read_csv("./data/t5_translated.csv")
t5_translated.head(3)

,en,es
0,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco sobre la irracion...
1,I was burned very badly.And if you spend a lot...,Me incendió muy mal.Y si pasamos mucho tiempo ...
2,The nurses in my department thought that the r...,Las enfermeras en mi departamento pensaban que...


In [5]:
t5_corrupted = pd.read_csv("./data/t5_corrupt_translated_cleaned.csv")
t5_corrupted.head(3)


,en_corruption_1,es_corruption_1,en_corruption_2,es_corruption_2,en_corruption_3,es_corruption_3
0,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco sobre la irracion...,I whon to talk too your to_day a little bit ab...,Quiero hablar también de la irracionalidad pre...,we mant to talk to you todya a little bit abou...,hace muchos aos hablarles de la irracionalidad...
1,I was burned very badly.And it you spend as lo...,Me incendiéndose muy mal.Y pasas mucho tiempo ...,I was bruned very badley.And is yuo spend an a...,Estaba enfermada muy mal.Y es yuo pasar mucho ...,I saw burned vrey badly.And if you spend the l...,V mal cado.Y si pasas el tiempo una vez si el ...
2,The nurses in my department thought that the r...,Las enfermeras en mi departamento pensaban que...,The nurses in my deportment throught the an ri...,Las enfermeras en mi deportión a través de una...,The nurses it yoy department thougt that the w...,Las enfermeras del departamento tuvieron que e...


In [14]:
byt5_corrupted = pd.read_csv("./data/byt5_corrupt_translated_clean.csv")
byt5_corrupted.head(3)

,en_corruption_1,es_corruption_1,en_corruption_2,es_corruption_2,en_corruption_3,es_corruption_3
0,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco de la irracionali...,I want to talk to you today a little bit about...,Hoy quiero hablarles un poco sobre la irracion...,I whant to talk to yow today a litel bit about...,Hoy quiero hablarles un poco sobre la irracion...
1,I was burned very badly.And if you spend a tan...,Me quemé muy mal.Y si pasan un poco de tiempo ...,ld was burned very badly.And if yuo spend a lo...,ld estaba quemado muy mal.Y si yuo pasa mucho ...,I ws burned very badly.And it yo spend a lot o...,Me quemé muy mal.Y pasa mucho tiempo en el hos...
2,The nurses in my department thought that the r...,Las enfermeras en mi departamento pensaban que...,The nurse's in my department thot that the wri...,La enfermera está en mi departamento diciendo ...,The nurses im my deportment thought but the ri...,Las enfermeras que pensaban en mi deporte pens...


## ByT5

In [15]:
# Reference Spanish (gold)
refs = byt5_translated["es"].astype(str).tolist()

# Load multilingual sentence embedding model
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Encode references once for efficiency
ref_emb = model.encode(refs, batch_size=64, convert_to_tensor=True)

# Corruption columns to evaluate
corruption_cols = ["es_corruption_1", "es_corruption_2", "es_corruption_3"]

# Compute similarity for every corruption column
for col in corruption_cols:
    preds = byt5_corrupted[col].astype(str).tolist()
    
    pred_emb = model.encode(preds, batch_size=64, convert_to_tensor=True)
    
    # cosine similarity (diagonal = row-by-row)
    sims = util.cos_sim(pred_emb, ref_emb).diagonal().cpu().numpy()
    
    # Store similarity scores
    sim_col = col.replace("es_", "sim_")
    byt5_corrupted[sim_col] = sims
# Show the first few rows to verify
byt5_corrupted[[
    "es_corruption_1", "sim_corruption_1",
    "es_corruption_2", "sim_corruption_2",
    "es_corruption_3", "sim_corruption_3",
]].head(10)


,es_corruption_1,sim_corruption_1,es_corruption_2,sim_corruption_2,es_corruption_3,sim_corruption_3
0,Hoy quiero hablarles un poco de la irracionali...,0.974677,Hoy quiero hablarles un poco sobre la irracion...,0.894131,Hoy quiero hablarles un poco sobre la irracion...,0.966152
1,Me quemé muy mal.Y si pasan un poco de tiempo ...,0.925170,ld estaba quemado muy mal.Y si yuo pasa mucho ...,0.915105,Me quemé muy mal.Y pasa mucho tiempo en el hos...,0.848602
2,Las enfermeras en mi departamento pensaban que...,0.970257,La enfermera está en mi departamento diciendo ...,0.582783,Las enfermeras que pensaban en mi deporte pens...,0.776676
3,Y porque había tenido el 70 % de mi cuerpo que...,0.792249,"Y como tenía el 70 % de mi cuerpo quemado, tom...",0.965480,"Y bekus hice el 70 % de mi cuerpo quemó, es de...",0.734766
4,"Y como pueden imaginar, odiaba ese momento de ...",0.976526,"Y como pueden imaginar, calenté ese momento de...",0.664678,"Y como pueden imaginar, odiaba ese momento de ...",0.976526
5,"Y trataría de razonar la razón y decir: ""¿No i...",0.888929,"Y trataba de razonar entonces y decía: ""¿Por q...",0.867536,"Y un trataría de razonarlos y decir: ""¿Por qué...",0.900424
6,"¿Por qué no lo tomamos un poco más tiempo, qui...",0.887350,"¿Por qué no tomamos un pequeño logro, quizás d...",0.884475,"¿Cómo no lo tomamos un poco más tiempo, quizás...",0.884834
7,Y las enfermeras me dijeron cosas.Me dijeron q...,0.943734,Y las enfermeras me dijeron dos cosas.Me dijer...,0.977006,Y la enfermera me dijo dos cosas.Me dijeron qu...,0.970384
8,eso fue lo que hice.,0.930089,Eso fue lo que hice.,0.930936,La caída sana es lo que hice.,0.703834
9,Todavía estaba interesada en esta pregunta de ...,0.978082,Todavía estaba interresada en esta pregunta de...,0.931157,Todavía estaba interesada por esa pregunta de ...,0.981156


In [6]:
# load data
df_ref = pd.read_csv('./data/t5_translated.csv')
df_hyp = pd.read_csv('./data/t5_corrupt_translated.csv')


def extract(txt):
    try:
        lst = ast.literal_eval(txt)
        if isinstance(lst, list):
            return " ".join(lst)
    except:
        pass
    return txt


# reference Spanish text
df_ref['es'] = df_ref['es'].apply(extract)

# corrupted Spanish lists -> strings
for col in ["es_corruption_1", "es_corruption_2", "es_corruption_3"]:
    df_hyp[col] = df_hyp[col].apply(extract)

# CHRF implementation


def character_ngrams(s, n):
    s = s.replace(" ", "")
    return [s[i:i+n] for i in range(len(s)-n+1)]


def chrf_score(ref, hyp, max_n=6, beta=1):
    total_prec, total_rec = 0.0, 0.0
    eps = 1e-16
    for n in range(1, max_n+1):
        ref_ngr = character_ngrams(ref, n)
        hyp_ngr = character_ngrams(hyp, n)
        ref_cnt = Counter(ref_ngr)
        hyp_cnt = Counter(hyp_ngr)
        overlap = sum(min(hyp_cnt[g], ref_cnt[g]) for g in hyp_cnt)
        prec = overlap / (len(hyp_ngr) + eps)
        rec = overlap / (len(ref_ngr) + eps)
        total_prec += prec
        total_rec += rec

    CHRP = total_prec / max_n
    CHRR = total_rec / max_n
    beta2 = beta * beta
    return (1 + beta2) * CHRP * CHRR / (beta2 * CHRP + CHRR + eps)


# store results
refs = df_ref['es'].tolist()

# clean baseline
bleu_clean = sacrebleu.corpus_bleu(refs, [refs]).score
chrf_clean = sum(chrf_score(r, r) for r in refs) / len(refs)

results = {"clean": {"BLEU": bleu_clean, "CHRF": 100 * chrf_clean}}

# corrupted versions
for i in [1, 2, 3]:
    hyps = df_hyp[f'es_corruption_{i}'].tolist()

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    chrf_vals = [chrf_score(ref, hyp) for ref, hyp in zip(refs, hyps)]

    results[f'corruption_{i}'] = {
        "BLEU": bleu,
        "CHRF": 100 * (sum(chrf_vals) / len(chrf_vals))
    }

print("= BLEU & CHRF Scores =\n")
print(f"{'Dataset':<15}{'BLEU':<12}{'CHRF'}")
print("-" * 35)

for key, vals in results.items():
    print(f"{key:<15}{vals['BLEU']:<12.4f}{vals['CHRF']:.4f}")


NameError: name 'sacrebleu' is not defined

In [ ]:
# load data
df_ref = pd.read_csv('./data/byt5_translated.csv')
df_hyp = pd.read_csv('./data/byt5_corrupt_translated.csv')


def extract(txt):
    try:
        lst = ast.literal_eval(txt)
        if isinstance(lst, list):
            return " ".join(lst)
    except:
        pass
    return txt


df_ref['es'] = df_ref['es'].apply(extract)

for col in ["es_corruption_1", "es_corruption_2", "es_corruption_3"]:
    df_hyp[col] = df_hyp[col].apply(extract)

# CHRF implementation


def character_ngrams(s, n):
    s = s.replace(" ", "")
    return [s[i:i+n] for i in range(len(s)-n+1)]


def chrf_score(ref, hyp, max_n=6, beta=1):
    total_prec, total_rec = 0.0, 0.0
    eps = 1e-16
    for n in range(1, max_n+1):
        ref_ngr = character_ngrams(ref, n)
        hyp_ngr = character_ngrams(hyp, n)
        ref_cnt = Counter(ref_ngr)
        hyp_cnt = Counter(hyp_ngr)
        overlap = sum(min(hyp_cnt[g], ref_cnt[g]) for g in hyp_cnt)
        prec = overlap / (len(hyp_ngr) + eps)
        rec = overlap / (len(ref_ngr) + eps)
        total_prec += prec
        total_rec += rec

    CHRP = total_prec / max_n
    CHRR = total_rec / max_n
    beta2 = beta * beta
    return (1 + beta2) * CHRP * CHRR / (beta2 * CHRP + CHRR + eps)


# Compute BLEU and CHRF scores
refs = df_ref['es'].tolist()

# Clean (self-BLEU)
bleu_clean = sacrebleu.corpus_bleu(refs, [refs]).score
chrf_clean = sum(chrf_score(r, r) for r in refs) / len(refs)
results = {"clean": {"BLEU": bleu_clean, "CHRF": 100 * chrf_clean}}

# Corruption levels
for i in [1, 2, 3]:
    hyps = df_hyp[f'es_corruption_{i}'].tolist()

    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    chrf_vals = [chrf_score(ref, hyp) for ref, hyp in zip(refs, hyps)]
    results[f'corruption_{i}'] = {
        "BLEU": bleu,
        "CHRF": 100 * sum(chrf_vals) / len(chrf_vals)
    }

print("= BLEU & CHRF Scores =\n")
print(f"{'Dataset':<15}{'BLEU':<12}{'CHRF'}")
print("-" * 35)

for key, vals in results.items():
    print(f"{key:<15}{vals['BLEU']:<12.4f}{vals['CHRF']:.4f}")


NameError: name 't5_eval' is not defined

In [ ]:
average_similarities_byt5 = {}

for col in corruption_cols:
    sim_col = col.replace("es_", "sim_")
    avg = byt5_corrupted[sim_col].mean()
    average_similarities_byt5[sim_col] = avg

average_similarities_byt5

{'sim_corruption_1': 0.9119863,
 'sim_corruption_2': 0.881925,
 'sim_corruption_3': 0.8456022}

In [ ]:
byt5_corrupted[["sim_corruption_1", "sim_corruption_2", "sim_corruption_3"]].std()

sim_corruption_1    0.084821
sim_corruption_2    0.103865
sim_corruption_3    0.122129
dtype: float32

## T5

In [13]:
# Reference Spanish (gold)
refs = t5_translated["es"].astype(str).tolist()

# Load multilingual sentence embedding model
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Encode references once for efficiency
ref_emb = model.encode(refs, batch_size=64, convert_to_tensor=True)

# Corruption columns to evaluate
corruption_cols = ["es_corruption_1", "es_corruption_2", "es_corruption_3"]

# Compute similarity for every corruption column
for col in corruption_cols:
    preds = t5_corrupted[col].astype(str).tolist()
    
    pred_emb = model.encode(preds, batch_size=64, convert_to_tensor=True)
    
    # cosine similarity (diagonal = row-by-row)
    sims = util.cos_sim(pred_emb, ref_emb).diagonal().cpu().numpy()
    
    # Store similarity scores
    sim_col = col.replace("es_", "sim_")
    t5_corrupted[sim_col] = sims

# Show the first few rows to verify
t5_corrupted[[
    "es_corruption_1", "sim_corruption_1",
    "es_corruption_2", "sim_corruption_2",
    "es_corruption_3", "sim_corruption_3",
]].head(10)


,es_corruption_1,sim_corruption_1,es_corruption_2,sim_corruption_2,es_corruption_3,sim_corruption_3
0,Hoy quiero hablarles un poco sobre la irracion...,1.000000,Quiero hablar también de la irracionalidad pre...,0.944967,hace muchos aos hablarles de la irracionalidad...,0.921071
1,Me incendiéndose muy mal.Y pasas mucho tiempo ...,0.955024,Estaba enfermada muy mal.Y es yuo pasar mucho ...,0.888758,V mal cado.Y si pasas el tiempo una vez si el ...,0.782693
2,Las enfermeras en mi departamento pensaban que...,0.986206,Las enfermeras en mi deportión a través de una...,0.659180,Las enfermeras del departamento tuvieron que e...,0.616414
3,Y como tena el 70 % de mi cuerpo incendiéndolo...,0.925898,"a porque tena el 70 % de mi cuerpo querrió, ll...",0.890106,Y en caso de que me incenda el 70 por ciento d...,0.652964
4,"Y como pueden imaginar, odiaba ese momento de ...",1.000000,"como imaginas, odiaba ese momento de correr co...",0.821959,"Y como se puede inmagnia, hice ese momento de ...",0.410001
5,"Y tratara de resonar con ellos y decir: ""Por q...",0.933644,"Y tratara de resonar ah as que digamos: ""No te...",0.713777,"y yo dije: ""Por qué no triamos algo eles?",0.880119
6,"Por qué no lo tomamos un poco más tiempo, pued...",0.958482,"Por qué no lo tomamos un poco más tiempo, quiz...",1.000000,"Por qué no lo ponemos un poco más tiempo, quiz...",0.918628
7,las enfermeras me dijeron dos cosas.Me dijeron...,0.951262,Y las enfermeras me dijeron dos cosas.Me dijer...,0.979228,Y las enfermeras me dijeron dos cosas.thrar te...,0.873300
8,As que eso fue lo que hice.,1.000000,As que eso fue lo que hice.,1.000000,As que vat es lo que hice.,0.955312
9,Todava estaba interesado en esta queston de có...,0.859765,Todava estaba interesado en esta pregunta de c...,0.910596,Todava estaba interesado en la pregunta de cóm...,0.634895


In [ ]:
average_similarities_t5 = {}

for col in corruption_cols:
    sim_col = col.replace("es_", "sim_")
    avg = t5_corrupted[sim_col].mean()
    average_similarities_t5[sim_col] = avg

average_similarities_t5

{'sim_corruption_1': 0.9376337,
 'sim_corruption_2': 0.8875044,
 'sim_corruption_3': 0.83936155}

In [ ]:
t5_corrupted[[
    "sim_corruption_1",
    "sim_corruption_2",
    "sim_corruption_3",
]].std()

sim_corruption_1    0.092200
sim_corruption_2    0.125807
sim_corruption_3    0.142845
dtype: float32